In [ ]:
from ultralytics import YOLO
import cv2
from facenet_pytorch import InceptionResnetV1, MTCNN
from services.embeddings import extract_embeddings, compare_embeddings
from services.utils import face_detection
from PIL import Image, ExifTags
import torch
from torchvision import transforms
import numpy as np

In [ ]:
resnet = InceptionResnetV1(pretrained='casia-webface').eval()
preprocess = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
])

def get_embeddings(face):
    if isinstance(face, np.ndarray):
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = Image.fromarray(face)
    
    face_tensor = preprocess(face)
    face_tensor = face_tensor.unsqueeze(0)
    
    with torch.no_grad():
        embeddings = resnet(face_tensor)
    
    return embeddings

In [ ]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, img = cap.read()
    
    if ret == True:
        count+=1
    else:
        continue
    
    if count == 30:
        break
    
cap.release()

In [ ]:
model = YOLO("Models/faceDetection/face_detection.pt")

results = model.predict(img, conf=0.4)

results = results[0]

bboxes = face_detection(results)

for box in bboxes:
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    face = img[int(y1):int(y2), int(x1):int(x2)]
    emb = get_embeddings(face)

In [ ]:
img = cv2.imread("reference4.jpg")

results = model.predict(img, conf=0.4)

results = results[0]

bboxes = face_detection(results)
print(bboxes)
for box in bboxes:
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    face = img[int(y1):int(y2), int(x1):int(x2)]
    emb2 = get_embeddings(face)

In [ ]:
print(compare_embeddings(emb, emb2))